In [1]:
import json
import sys
import yaml
import re
# List of entity types we want to evaluate
ENTITY_TYPES = [
    "citation",
    "judges",
    "ConvCourtName",
    "ConvictPleaDate",
    "ConvictOffence",
    "AcquitOffence",
    "ConfessPleadGuilty",  # Adjusted entity name to match YAML sample
    "PleaPoint",
    "RemandDecision",
    "RemandCustodyTime",
    "SentCourtName",
    "SentenceReceived",
    "SentenceServed",
    "WhatAncilliary",
    "OffSex",
    "OffAgeOffence",
    "OffJobOffence",
    "OffHomeOffence",
    "OffMentalOffence",
    "OffIntoxOffence",
    "OffVicRelation",
    "VictimType",
    "VicNum",
    "VicSex",
    "VicAgeOffence",
    "VicJobOffence",
    "VicHomeOffence",
    "VicMentalOffence",
    "VicIntoxOffence",
    "ProsEvidTypeTrial",
    "DefEvidTypeTrial",
    "PreSentReport",
    "AggFactSent",
    "MitFactSent",
    "VicImpactStatement",
    "Appellant",
    "CoDefAccNum",
    "AppealAgainst",
    "AppealGround",
    "SentGuideWhich",
    "AppealOutcome",
    "ReasonQuashConv",
    "ReasonSentExcessNotLenient",
    "ReasonSentLenientNotExcess",
    "ReasonDismiss",
]

def is_empty(val):
    """Determine if a value is effectively empty or non-informative."""
    if val is None:
        return True
    if isinstance(val, str):
        cleaned = val.strip().lower()
        if cleaned in ["", "nan", "null", "-"]:
            return True
    return False

def compute_metrics_from_counts(tp, fp, fn):
    """Compute precision, recall, and F1 given counts of TP, FP, FN."""
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    return precision, recall, f1


import re

def parse_custom_text(input_text):
    """
    Parse text into structured data, handling variations and edge cases.
    """
    # Split into sections using `\n` as a logical delimiter
    sections = input_text.split("\n")
    structured_data = []
    current_section = {}
    current_key = None

    # Patterns to identify section headers, Comment, and Exact fields
    section_pattern = re.compile(r"^(\w+):$")
    comment_pattern = re.compile(r"^\s*Comment:\s*(.*)$")
    exact_pattern = re.compile(r"^\s*Exact:\s*(.*)$")

    for line in sections:
        # Match section header (e.g., `citation:`)
        section_match = section_pattern.match(line.strip())
        if section_match:
            # Save the current section if it exists
            if current_section:
                structured_data.append(current_section)
                current_section = {}
            # Start a new section
            current_key = section_match.group(1)
            current_section["section"] = current_key
            continue

        # Match Comment field
        comment_match = comment_pattern.match(line.strip())
        if comment_match:
            comment_value = comment_match.group(1).strip("'").strip('"') or "nan"
            current_section["Comment"] = comment_value
            continue

        # Match Exact field
        exact_match = exact_pattern.match(line.strip())
        if exact_match:
            exact_value = exact_match.group(1).strip("'").strip('"') or "nan"
            current_section["Exact"] = exact_value
            continue

    # Append the last section if it exists
    if current_section:
        structured_data.append(current_section)

    return structured_data




In [2]:
entity_stats = { entity: {"TP": 0, "FP": 0, "FN": 0} for entity in ENTITY_TYPES }

total_tp = total_fp = total_fn = 0


json_file_path='/home/stirunag/Downloads/sft_on_english_data/sft_on_english_data/predictions/llama_3.1_8b_instruct_fine_tuned_appealcourt_coded/outputs_42.json'


with open(json_file_path, "r", encoding="utf-8") as f:
    data = json.load(f)


In [4]:
len(data)

125

In [19]:
answer_text = data[10].get("answer", "")
gold_text = data[10].get("gold", "")

[answer_text, gold_text]

['```yaml\ncitation:\n  Comment: \'[2023] EWCA Crim 1252\'\n  Exact: \'IN THE COURT OF APPEAL CRIMINAL DIVISION [2023] EWCA Crim 1252 No. 202300471 A4 Royal Courts of Justice Wednesday, 11 October 2023\'\n\njudges:\n  Comment: \'LORD JUSTICE WARBY, MR JUSTICE MURRAY, HIS HONOUR JUDGE LEONARD KC\'\n  Exact: \'Before: LORD JUSTICE WARBY MR JUSTICE MURRAY HIS HONOUR JUDGE LEONARD KC\'\n\nConvCourtName:\n  Comment: \'Crown Court at Leeds\'\n  Exact: \'On 30 January 2023, in the Crown Court at Leeds\'\n\nConvictPleaDate:\n  Comment: \'2023-01-30\'\n  Exact: \'On 30 January 2023\'\n\nConvictOffence:\n  Comment: \'Participation in a conspiracy to supply a class A drug, supplying a class A drug, dangerous driving, perverting the course of justice\'\n  Exact: \'The offences we have mentioned were the subject of three indictments. On the first indictment  (case T20217672), the appellant faced a single count of participation in a conspiracy to supply a class A drug between 1 May and 1 October 202

In [20]:
answer_parsed = parse_custom_text(answer_text) 
answer_parsed

[{'section': 'citation',
  'Comment': '[2023] EWCA Crim 1252',
  'Exact': 'IN THE COURT OF APPEAL CRIMINAL DIVISION [2023] EWCA Crim 1252 No. 202300471 A4 Royal Courts of Justice Wednesday, 11 October 2023'},
 {'section': 'judges',
  'Comment': 'LORD JUSTICE WARBY, MR JUSTICE MURRAY, HIS HONOUR JUDGE LEONARD KC',
  'Exact': 'Before: LORD JUSTICE WARBY MR JUSTICE MURRAY HIS HONOUR JUDGE LEONARD KC'},
 {'section': 'ConvCourtName',
  'Comment': 'Crown Court at Leeds',
  'Exact': 'On 30 January 2023, in the Crown Court at Leeds'},
 {'section': 'ConvictPleaDate',
  'Comment': '2023-01-30',
  'Exact': 'On 30 January 2023'},
 {'section': 'ConvictOffence',
  'Comment': 'Participation in a conspiracy to supply a class A drug, supplying a class A drug, dangerous driving, perverting the course of justice',
  'Exact': 'The offences we have mentioned were the subject of three indictments. On the first indictment  (case T20217672), the appellant faced a single count of participation in a conspiracy 

In [21]:
gold_parsed = parse_custom_text(gold_text)
gold_parsed

[{'section': 'citation', 'Comment': 'nan', 'Exact': '[2023] EWCA Crim 1252'},
 {'section': 'judges',
  'Comment': 'nan',
  'Exact': "['LORD JUSTICE WARBY', 'MR JUSTICE MURRAY', 'HIS HONOUR JUDGE LEONARD KC']"},
 {'section': 'ConvCourtName',
  'Comment': 'nan',
  'Exact': 'Crown Court at Leeds'},
 {'section': 'ConvictPleaDate', 'Comment': 'nan', 'Exact': '30 January 2023'},
 {'section': 'ConvictOffence',
  'Comment': 'nan',
  'Exact': 'supplying a class A drug'},
 {'section': 'AcquitOffence', 'Comment': 'nan', 'Exact': 'nan'},
 {'section': 'PleaPoint',
  'Comment': 'nan',
  'Exact': 'at the plea and trial preparation hearing'},
 {'section': 'RemandDecision', 'Comment': 'nan', 'Exact': 'on remand'},
 {'section': 'RemandCustodyTime', 'Comment': 'nan', 'Exact': "Don't know"},
 {'section': 'SentCourtName',
  'Comment': 'nan',
  'Exact': 'Crown Court at Leeds'},
 {'section': 'Sentence',
  'Comment': 'nan',
  'Exact': "seven years and four months' imprisonment"},
 {'section': 'SentServe', 'Co

In [23]:
from difflib import SequenceMatcher

def similarity(a, b):
    """Compute similarity ratio between two strings."""
    return SequenceMatcher(None, a, b).ratio()

SIMILARITY_THRESHOLD = 0.5  # Adjust based on acceptable similarity level

# Process each record
for record in data:
    answer_text = record.get("answer", "")
    gold_text = record.get("gold", "")

    # Parse the structured text from answer and gold fields
    try:
        answer_parsed = parse_custom_text(answer_text) or []
    except Exception as e:
        print(f"Error parsing answer text: {e}")
        answer_parsed = []
    try:
        gold_parsed = parse_custom_text(gold_text) or []
    except Exception as e:
        print(f"Error parsing gold text: {e}")
        gold_parsed = []

    # Convert parsed lists into dictionaries for easier lookup
    answer_dict = {entry["section"]: entry for entry in answer_parsed}
    gold_dict = {entry["section"]: entry for entry in gold_parsed}

    for entity in ENTITY_TYPES:
        # Extract "Comment" and "Exact" values
        gold_entity = gold_dict.get(entity, {})
        pred_entity = answer_dict.get(entity, {})
    
        gold_comment = gold_entity.get("Comment", "nan")
        pred_comment = pred_entity.get("Comment", "nan")
    
        gold_exact = gold_entity.get("Exact", "nan")
        pred_exact = pred_entity.get("Exact", "nan")
    
        # Initialize stats for the entity
        if entity not in entity_stats:
            entity_stats[entity] = {"TP": 0, "FP": 0, "FN": 0}
    
        # Function to update TP, FP, FN based on match
        def update_metrics(gold, pred):
            gold_present = not is_empty(gold)
            pred_present = not is_empty(pred)
            if gold_present:
                if pred_present:
                    # Check for similarity
                    if similarity(str(gold).strip(), str(pred).strip()) >= SIMILARITY_THRESHOLD:
                        entity_stats[entity]["TP"] += 1
                        return 1, 0, 0  # TP
                    else:
                        # Similarity below threshold
                        return 0, 1, 1  # FP, FN
                else:
                    # Gold present but prediction missing
                    return 0, 0, 1  # FN
            else:
                if pred_present:
                    # Prediction made but nothing in gold
                    return 0, 1, 0  # FP
                # Neither present: do nothing
                return 0, 0, 0
    
        # Update metrics for "Comment"
        tp, fp, fn = update_metrics(gold_comment, pred_comment)
        entity_stats[entity]["TP"] += tp
        entity_stats[entity]["FP"] += fp
        entity_stats[entity]["FN"] += fn
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
        # Update metrics for "Exact"
        tp, fp, fn = update_metrics(gold_exact, pred_exact)
        entity_stats[entity]["TP"] += tp
        entity_stats[entity]["FP"] += fp
        entity_stats[entity]["FN"] += fn
        total_tp += tp
        total_fp += fp
        total_fn += fn

# Compute overall micro-averaged metrics
overall_precision, overall_recall, overall_f1 = compute_metrics_from_counts(total_tp, total_fp, total_fn)
print("\nOverall (Micro-Averaged) Metrics:")
print(f"Precision = {overall_precision:.3f}")
print(f"Recall    = {overall_recall:.3f}")
print(f"F1        = {overall_f1:.3f}")



Overall (Micro-Averaged) Metrics:
Precision = 0.109
Recall    = 0.199
F1        = 0.141


In [25]:
import pandas as pd

# Prepare data structure for storing results per citation
results = {}

# Process each record in data
for record in data:
    answer_text = record.get("answer", "")
    gold_text = record.get("gold", "")

    # Parse structured content from answer and gold fields
    try:
        answer_parsed = parse_custom_text(answer_text) or []
    except Exception as e:
        print(f"Error parsing answer text: {e}")
        answer_parsed = []

    try:
        gold_parsed = parse_custom_text(gold_text) or []
    except Exception as e:
        print(f"Error parsing gold text: {e}")
        gold_parsed = []

    # Convert parsed lists to dictionaries for easier lookup
    answer_dict = {entry["section"]: entry for entry in answer_parsed}
    gold_dict = {entry["section"]: entry for entry in gold_parsed}

    # Extract citation for column grouping
    citation_entity = gold_dict.get("citation", {})
    citation_number = citation_entity.get("Exact", "Unknown Citation")

    if citation_number not in results:
        results[citation_number] = []

    for entity in ENTITY_TYPES:
        # Extract gold and predicted values for exact and comment
        gold_entity = gold_dict.get(entity, {})
        pred_entity = answer_dict.get(entity, {})

        gold_exact = gold_entity.get("Exact", "nan")
        gold_comment = gold_entity.get("Comment", "nan")
        pred_exact = pred_entity.get("Exact", "nan")
        pred_comment = pred_entity.get("Comment", "nan")

        # Combine exact and comment for display in a single cell
        human_combined = f"EXACT: {gold_exact}\nCOMMENT: {gold_comment}"
        ai_combined = f"EXACT: {pred_exact}\nCOMMENT: {pred_comment}"

        # Add results to the list for this citation
        results[citation_number].append({
            "Entity Type": entity,
            "Human": human_combined,
            "AI": ai_combined,
        })

# Create a DataFrame for structured evaluation output
rows = []
columns = set()

for citation, entities in results.items():
    for entity_data in entities:
        entity_type = entity_data["Entity Type"]
        human_value = entity_data["Human"]
        ai_value = entity_data["AI"]

        # Ensure the row exists for the current entity type
        row = next((r for r in rows if r["Entity Type"] == entity_type), None)
        if not row:
            row = {"Entity Type": entity_type}
            rows.append(row)

        # Add human and AI data under the citation's subcolumns
        row[f"{citation} - Human"] = human_value
        row[f"{citation} - AI"] = ai_value

# Convert to DataFrame
df_citation_eval = pd.DataFrame(rows)

# Save the DataFrame as a CSV
output_csv_path = "data/evaluation_by_citation.csv"
df_citation_eval.to_csv(output_csv_path, index=False)

output_csv_path


'data/evaluation_by_citation.csv'

In [26]:
df_citation_eval

,Entity Type,[2024] EWCA Crim 474 - Human,[2024] EWCA Crim 474 - AI,[2014] EWCA Crim 1594 - Human,[2014] EWCA Crim 1594 - AI,[2017] EWCA Crim 849 - Human,[2017] EWCA Crim 849 - AI,[2015] EWCA Crim 1791 - Human,[2015] EWCA Crim 1791 - AI,[2007] EWCA Crim 3312 - Human,...,[2016] EWCA Crim 1841 - Human,[2016] EWCA Crim 1841 - AI,[2019] EWCA Crim 1628 - Human,[2019] EWCA Crim 1628 - AI,[2007] EWCA Crim 2787 - Human,[2007] EWCA Crim 2787 - AI,[2004] EWCA Crim 2901 - Human,[2004] EWCA Crim 2901 - AI,[2018] EWCA Crim 1374 - Human,[2018] EWCA Crim 1374 - AI
0,citation,EXACT: [2024] EWCA Crim 474\nCOMMENT: nan,"EXACT: On 24 April 2024, Before: LORD JUSTICE ...",EXACT: [2014] EWCA Crim 1594\nCOMMENT: nan,"EXACT: On 12th June 2014, this court, differen...",EXACT: [2017] EWCA Crim 849\nCOMMENT: nan,EXACT: IN THE COURT OF APPEAL CRIMINAL DIVISIO...,EXACT: [2015] EWCA Crim 1791\nCOMMENT: nan,EXACT: IN THE COURT OF APPEAL (CRIMINAL DIVISI...,EXACT: [2007] EWCA Crim 3312\nCOMMENT: nan,...,EXACT: [2016] EWCA Crim 1841\nCOMMENT: nan,EXACT: On APPEAL from the Crown Court at Notti...,EXACT: [2019] EWCA Crim 1628\nCOMMENT: nan,EXACT: NCN: [2019] EWCA (Crim) 1628\nCOMMENT: ...,EXACT: [2007] EWCA Crim 2787\nCOMMENT: nan,EXACT: Neutral Citation Number: [2007] EWCA Cr...,EXACT: [2004] EWCA Crim 2901\nCOMMENT: nan,EXACT: Neutral Citation Number: [2004] EWCA Cr...,EXACT: [2018] EWCA Crim 1374\nCOMMENT: nan,"EXACT: On 22 January 2018, the appellant, Chri..."
1,judges,"EXACT: ['LORD JUSTICE MALES', 'MR JUSTICE HILL...",EXACT: Before: LORD JUSTICE MALES MR JUSTICE H...,"EXACT: ['MR JUSTICE GLOBE', 'SIR RODERICK EVAN...",EXACT: PRESIDENT OF THE QUEEN\'S BENCH DIVISIO...,"EXACT: ['LORD JUSTICE HICKINBOTTOM', 'MR JUSTI...",EXACT: LORD JUSTICE HICKINBOTTOM MR JUSTICE HO...,"EXACT: ['LADY JUSTICE RAFFERTY', 'MR JUSTICE H...",EXACT: Before: LADY JUSTICE RAFFERTY MR JUSTIC...,"EXACT: ['LORD JUSTICE LAWS', 'MR JUSTICE MACKA...",...,EXACT: ['MR JUSTICE MALES']\nCOMMENT: nan,EXACT: THE PRESIDENT OF THE QUEEN’S BENCH DIVI...,"EXACT: ['LORD JUSTICE SIMON', 'MRS JUSTICE McG...",EXACT: B e f o r e : LORD JUSTICE SIMON MRS JU...,"EXACT: ['LORD JUSTICE LATHAM', 'MR JUSTICE AIK...",EXACT: B e f o r e : LORD JUSTICE LATHAM Vice ...,"EXACT: ['LORD JUSTICE CLARKE', 'MR JUSTICE DOU...",EXACT: B E F O R E: LORD JUSTICE CLARKE MR JUS...,"EXACT: ['LORD JUSTICE SIMON', 'MR JUSTICE GOOS...",EXACT: B e f o r e : LORD JUSTICE SIMON MR JUS...
2,ConvCourtName,EXACT: nan\nCOMMENT: nan,"EXACT: On 18 October 2022, in the Crown Court ...",EXACT: Crown Court at Inner London\nCOMMENT: nan,EXACT: On 14th June 2011 in the Crown Court at...,EXACT: nan\nCOMMENT: nan,EXACT: On 18th August 2016 in the Crown Court ...,EXACT: nan\nCOMMENT: nan,EXACT: ON APPEAL FROM CROWN COURT AT WOOLWICH\...,EXACT: Harrow Crown Court\nCOMMENT: nan,...,EXACT: nan\nCOMMENT: nan,EXACT: IN THE COURT OF APPEAL (CRIMINAL DIVISI...,EXACT: nan\nCOMMENT: nan,EXACT: On 8 March 2019 the appellant pleaded g...,EXACT: nan\nCOMMENT: nan,EXACT: On 3rd March in the Crown Court at Tees...,EXACT: nan\nCOMMENT: nan,"EXACT: On 1st December 2003, in the Crown Cour...",EXACT: Crown Court at Stoke-on-Trent\nCOMMENT:...,EXACT: He was sentenced in the Crown Court at ...
3,ConvictPleaDate,EXACT: nan\nCOMMENT: nan,"EXACT: On 18 October 2022, in the Crown Court ...",EXACT: 14th June 2011\nCOMMENT: nan,EXACT: On 14th June 2011 in the Crown Court at...,EXACT: nan\nCOMMENT: nan,EXACT: On 18th August 2016\nCOMMENT: 2016-08-18,EXACT: nan\nCOMMENT: nan,EXACT: On 12 October 2012\nCOMMENT: 2012-10-12,EXACT: 30 August 2007\nCOMMENT: nan,...,EXACT: nan\nCOMMENT: nan,EXACT: On 4 November 2015\nCOMMENT: 2015-11-04,EXACT: nan\nCOMMENT: nan,EXACT: On 8 March 2019 the appellant pleaded g...,EXACT: nan\nCOMMENT: nan,EXACT: On 3rd March in the Crown Court at Tees...,EXACT: nan\nCOMMENT: nan,EXACT: On 1st December 2003\nCOMMENT: 2003-12-01,EXACT: 22 January 2018\nCOMMENT: nan,EXACT: On 22 January 2018\nCOMMENT: 2018-01-22
4,ConvictOffence,EXACT: nan\nCOMMENT: 

In [6]:
import json
import sys
import yaml
import re
# List of entity types we want to evaluate
ENTITY_TYPES = [
    "citation",
    "judges",
    "ConvCourtName",
    "ConvictPleaDate",
    "ConvictOffence",
    "AcquitOffence",
    "ConfessPleadGuilty",  # Adjusted entity name to match YAML sample
    "PleaPoint",
    "RemandDecision",
    "RemandCustodyTime",
    "SentCourtName",
    "SentenceReceived",
    "SentenceServed",
    "WhatAncilliary",
    "OffSex",
    "OffAgeOffence",
    "OffJobOffence",
    "OffHomeOffence",
    "OffMentalOffence",
    "OffIntoxOffence",
    "OffVicRelation",
    "VictimType",
    "VicNum",
    "VicSex",
    "VicAgeOffence",
    "VicJobOffence",
    "VicHomeOffence",
    "VicMentalOffence",
    "VicIntoxOffence",
    "ProsEvidTypeTrial",
    "DefEvidTypeTrial",
    "PreSentReport",
    "AggFactSent",
    "MitFactSent",
    "VicImpactStatement",
    "Appellant",
    "CoDefAccNum",
    "AppealAgainst",
    "AppealGround",
    "SentGuideWhich",
    "AppealOutcome",
    "ReasonQuashConv",
    "ReasonSentExcessNotLenient",
    "ReasonSentLenientNotExcess",
    "ReasonDismiss",
]

def is_empty(val):
    """Determine if a value is effectively empty or non-informative."""
    if val is None:
        return True
    if isinstance(val, str):
        cleaned = val.strip().lower()
        if cleaned in ["", "nan", "null", "-"]:
            return True
    return False

def compute_metrics_from_counts(tp, fp, fn):
    """Compute precision, recall, and F1 given counts of TP, FP, FN."""
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    return precision, recall, f1

def fix_single_quotes(yaml_text):
    """
    Fix single-quote and scalar issues in YAML, ensuring proper quoting and escaping.
    """
    # Fix improperly quoted strings
    pattern = re.compile(r"(^\s*[^:]+:\s*)'(.*?)'(\s*(?:\n|$))", re.MULTILINE)
    def replacer(match):
        prefix, content, suffix = match.groups()
        fixed_content = content.replace("'", "''")  # Escape single quotes
        return f"{prefix}'''{fixed_content}'''{suffix}"
    fixed_text = pattern.sub(replacer, yaml_text)

    # Ensure all unquoted scalar values are wrapped in double quotes
    pattern_unquoted = re.compile(r"(^\s*[^:]+:\s*)([^\s\n\"'][^\n]*)(\n|$)", re.MULTILINE)
    def wrap_unquoted(match):
        prefix, value, suffix = match.groups()
        return f'{prefix}"{value.strip()}"{suffix}'
    fixed_text = pattern_unquoted.sub(wrap_unquoted, fixed_text)

    return fixed_text


def preprocess_yaml(yaml_text):
    """
    Fix common issues in YAML for parsing, ensuring proper quoting and escaping.
    """
    # Fix misplaced or dangling quotes
    yaml_text = re.sub(r'"([^"]*)"\s+([^"\n]+)(?=\n|$)', r'"\1 \2"', yaml_text)

    # Escape single and double quotes within strings
    yaml_text = yaml_text.replace("'", "''").replace('"', '\\"')

    return yaml_text


def parse_yaml_content(text):
    """
    Parse YAML with fixes for quoting and syntax issues.
    """
    # Remove YAML fences if present
    if text.startswith("```yaml"):
        text = text.replace("```yaml", "", 1)
    if text.endswith("```"):
        text = text[:-len("```")]

    # Fix syntax issues
    text = preprocess_yaml(text)
    text = fix_single_quotes(text)

    # Parse the corrected YAML
    yaml_data = yaml.safe_load(text)

    # Replace empty values with "nan"
    def replace_empty(data):
        if isinstance(data, dict):
            return {k: replace_empty(v) for k, v in data.items()}
        elif isinstance(data, list):
            return [replace_empty(item) for item in data]
        elif is_empty(data):
            return "nan"
        else:
            return data

    return replace_empty(yaml_data)


# def fix_single_quotes(yaml_text):
#     """
#     Fix single-quote issues in YAML and quote unquoted Exact values.
#     """
#     # Fix single-quoted strings
#     pattern = re.compile(r"(^\s*[^:]+:\s*)'(.*?)'(\s*(?:\n|$))", re.MULTILINE)
    
#     def replacer(match):
#         prefix = match.group(1)
#         content = match.group(2)
#         suffix = match.group(3)
#         fixed_content = content.replace("'", "''")
#         return f"{prefix}'{fixed_content}'{suffix}"
    
#     fixed_text = pattern.sub(replacer, yaml_text)
    
#     # Wrap unquoted Exact: values in double quotes to handle problematic characters
#     pattern_exact = re.compile(r"(^\s*Exact:\s*)([^\n]+)(\n|$)", re.MULTILINE)
    
#     def replacer_exact(match):
#         prefix = match.group(1)
#         value = match.group(2)
#         suffix = match.group(3)
#         # If value isn't already quoted, wrap it in quotes
#         if not (value.startswith('"') or value.startswith("'")):
#             fixed_value = value.replace("'", "''")  # double single quotes inside value
#             return f'{prefix}"{fixed_value}"{suffix}'
#         else:
#             return match.group(0)
    
#     fixed_text = pattern_exact.sub(replacer_exact, fixed_text)
#     return fixed_text

# def parse_yaml_content(text):
#     """
#     Parse YAML from the provided text, fixing quote issues and standardizing empty values to 'nan'.
#     """
#     # Remove markdown fences if present
#     if text.startswith("```yaml"):
#         text = text.replace("```yaml", "", 1)
#     if text.endswith("```"):
#         text = text[:-len("```")]

#     # Preprocess YAML to fix common syntax issues
#     text = preprocess_yaml(text)

#     # Fix single-quote issues
#     text_fixed = fix_single_quotes(text)

#     # Parse the corrected YAML content
#     yaml_data = yaml.safe_load(text_fixed)

#     # Standardize empty or dash values to 'nan'
#     def standardize_values(data):
#         if isinstance(data, dict):
#             return {
#                 key: standardize_values(value)
#                 for key, value in data.items()
#             }
#         elif isinstance(data, list):
#             return [standardize_values(item) for item in data]
#         elif data in ["-", ""]:
#             return "nan"
#         else:
#             return data

#     return standardize_values(yaml_data)

# def preprocess_yaml(yaml_text):
#     """
#     Fix common issues in YAML content for parsing, including misplaced or missing quotes.
#     """
#     # Ensure all scalar values are properly quoted
#     yaml_text = re.sub(r'(?<=:\s)([^\s"\'][^\n]*[^\s"\'])(?=\n|$)', r'"\1"', yaml_text)

#     # Fix misplaced quotes by combining partial quoted strings
#     yaml_text = re.sub(r'"([^"]*)"\s+([^"\n]+)(?=\n|$)', r'"\1 \2"', yaml_text)

#     # Escape double quotes within quoted strings
#     yaml_text = re.sub(r'(?<!\\)"', r'\\"', yaml_text)

#     return yaml_text



In [11]:
!pip install ruamel.yaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 3.2 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 16.3 MB/s eta 0:00:00 MB/s eta 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [13]:
import re

def parse_custom_text(input_text):
    """
    Parse the provided string into structured data using specific delimiters and patterns.
    """
    # Split into sections based on `\n` and look for patterns with 'Comment:' and 'Exact:'
    sections = input_text.split("\n")
    structured_data = []
    current_section = {}
    current_key = None

    for line in sections:
        # Check for section header (e.g., citation:)
        section_match = re.match(r"^(\w+):$", line.strip())
        if section_match:
            # Save the current section if it exists
            if current_section:
                structured_data.append(current_section)
                current_section = {}
            # Start a new section
            current_key = section_match.group(1)
            current_section["section"] = current_key
            continue

        # Check for Comment line
        comment_match = re.match(r"^\s*Comment:\s*(.*)$", line.strip())
        if comment_match:
            current_section["Comment"] = comment_match.group(1).strip("'") or "nan"
            continue

        # Check for Exact line
        exact_match = re.match(r"^\s*Exact:\s*(.*)$", line.strip())
        if exact_match:
            current_section["Exact"] = exact_match.group(1).strip("'") or "nan"
            continue

    # Append the last section if it exists
    if current_section:
        structured_data.append(current_section)

    return structured_data


# Example input
input_text = """
citation:
  Comment: '[2024] EWCA Crim 474'
  Exact: 'On 24 April 2024, Before: LORD JUSTICE MALES MR JUSTICE HILLIARD RECORDER OF NORTHAMPTON (HIS HONOUR JUDGE MAYO) (Sitting as a Judge of the CACD) REX V RAJAK MIAH'

judges:
  Comment: 'LORD JUSTICE MALES, MR JUSTICE HILLIARD, RECORDER OF NORTHAMPTON (HIS HONOUR JUDGE MAYO)'
  Exact: 'Before: LORD JUSTICE MALES MR JUSTICE HILLIARD RECORDER OF NORTHAMPTON (HIS HONOUR JUDGE MAYO) (Sitting as a Judge of the CACD)'
"""

# Parse the text
parsed_data = parse_custom_text(input_text)

# Print the output
print(parsed_data)


[{'section': 'citation', 'Comment': '[2024] EWCA Crim 474', 'Exact': 'On 24 April 2024, Before: LORD JUSTICE MALES MR JUSTICE HILLIARD RECORDER OF NORTHAMPTON (HIS HONOUR JUDGE MAYO) (Sitting as a Judge of the CACD) REX V RAJAK MIAH'}, {'section': 'judges', 'Comment': 'LORD JUSTICE MALES, MR JUSTICE HILLIARD, RECORDER OF NORTHAMPTON (HIS HONOUR JUDGE MAYO)', 'Exact': 'Before: LORD JUSTICE MALES MR JUSTICE HILLIARD RECORDER OF NORTHAMPTON (HIS HONOUR JUDGE MAYO) (Sitting as a Judge of the CACD)'}]


In [7]:
entity_stats = { entity: {"TP": 0, "FP": 0, "FN": 0} for entity in ENTITY_TYPES }

total_tp = total_fp = total_fn = 0


json_file_path='/home/stirunag/Downloads/sft_on_english_data/sft_on_english_data/predictions/llama_3.1_8b_instruct_fine_tuned_appealcourt_coded/outputs_42.json'


with open(json_file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

data[1]

{'answer': '```yaml\ncitation:\n  Comment: \'[2014] EWCA Crim 1594\'\n  Exact: \'On 12th June 2014, this court, differently constituted, considered a Reference by the Registrar of Criminal Appeals, pursuant to section 20 of the Criminal Appeal Act 1968, to make a summary determination whether an application for leave to appeal a confiscation order should be dismissed as frivolous or vexatious.\'\n\njudges:\n  Comment: \'PRESIDENT OF THE QUEEN\\\'S BENCH DIVISION (SIR BRIAN LEVESON), MR JUSTICE GLOBE, SIR RODERICK EVANS\'\n  Exact: \'PRESIDENT OF THE QUEEN\\\'S BENCH DIVISION: 1. On 12th June 2014, this court, differently constituted, considered a Reference by the Registrar of Criminal Appeals, pursuant to section 20 of the Criminal Appeal Act 1968, to make a summary determination whether an application for leave to appeal a confiscation order should be dismissed as frivolous or vexatious.\'\n\nConvCourtName:\n  Comment: \'Crown Court at Inner London, Stockport Magistrates\\\' Court\'\n

In [10]:
answer_text = data[1].get("answer", "")
gold_text = data[1].get("gold", "")

answer_yaml = parse_yaml_content(answer_text) or {}
gold_yaml = parse_yaml_content(gold_text) or {}

ParserError: while parsing a block mapping
  in "<unicode string>", line 2, column 1:
    citation:
    ^
expected <block end>, but found '<block mapping start>'
  in "<unicode string>", line 4, column 3:
      Exact: '''''On 12th June 2014, t ... 
      ^

In [ ]:
answer_yaml

In [7]:
gold_yaml


{'citation': {'Comment': 'nan', 'Exact': '[2014] EWCA Crim 1594'},
 'judges': {'Comment': 'nan',
  'Exact': "[''MR JUSTICE GLOBE'', ''SIR RODERICK EVANS'']",
  'ConvCourtName': {'Comment': 'nan', 'Exact': 'Crown Court at Inner London'},
  'ConvictPleaDate': {'Comment': 'nan', 'Exact': '14th June 2011'},
  'ConvictOffence': {'Comment': 'nan',
   'Exact': 'conspiracy to supply controlled drugs of Class A'},
  'AcquitOffence': {'Comment': 'nan', 'Exact': 'nan'},
  'Confess/Plead': {'Comment': 'nan', 'Exact': 'nan'},
  'PleaPoint': {'Comment': 'nan', 'Exact': 'nan'},
  'RemandDecision': {'Comment': 'nan', 'Exact': 'nan'},
  'RemandCustodyTime': {'Comment': 'nan', 'Exact': 'nan'},
  'SentCourtName': {'Comment': 'nan', 'Exact': 'Crown Court at Inner London'},
  'Sentence': {'Comment': 'nan', 'Exact': 'nan'},
  'SentServe': {'Comment': 'nan', 'Exact': 'nan'},
  'WhatAncilliary': {'Comment': 'nan', 'Exact': 'nan'},
  'OffSex': {'Comment': 'male', 'Exact': 'Vincent Graham,'},
  'OffAgeOffence':

In [8]:
from difflib import SequenceMatcher

def similarity(a, b):
    """Compute similarity ratio between two strings."""
    return SequenceMatcher(None, a, b).ratio()

SIMILARITY_THRESHOLD = 0.5  # Adjust based on acceptable similarity level




In [18]:
# Process each record
for record in data:
    answer_text = record.get("answer", "")
    gold_text = record.get("gold", "")

    # Parse YAML content from answer and gold fields
    try:
        answer_yaml = parse_yaml_content(answer_text) or {}
    except Exception as e:
        print(f"Error parsing answer YAML: {e}")
        answer_yaml = {}
    try:
        gold_yaml = parse_yaml_content(gold_text) or {}
    except Exception as e:
        print(f"Error parsing gold YAML: {e}")
        gold_yaml = {}

    for entity in ENTITY_TYPES:
        # Extract "Comment" and "Exact" values
        gold_entity = gold_yaml.get(entity, {}) if isinstance(gold_yaml, dict) else {}
        pred_entity = answer_yaml.get(entity, {}) if isinstance(answer_yaml, dict) else {}
    
        gold_comment = gold_entity.get("Comment") if isinstance(gold_entity, dict) else None
        pred_comment = pred_entity.get("Comment") if isinstance(pred_entity, dict) else None
    
        gold_exact = gold_entity.get("Exact") if isinstance(gold_entity, dict) else None
        pred_exact = pred_entity.get("Exact") if isinstance(pred_entity, dict) else None
    
        # Initialize stats for the entity
        if entity not in entity_stats:
            entity_stats[entity] = {"TP": 0, "FP": 0, "FN": 0}
    
        # Function to update TP, FP, FN based on match
        def update_metrics(gold, pred):
            gold_present = not is_empty(gold)
            pred_present = not is_empty(pred)
            if gold_present:
                if pred_present:
                    # Check for similarity
                    if similarity(str(gold).strip(), str(pred).strip()) >= SIMILARITY_THRESHOLD:
                        entity_stats[entity]["TP"] += 1
                        return 1, 0, 0  # TP
                    else:
                        # Similarity below threshold
                        return 0, 1, 1  # FP, FN
                else:
                    # Gold present but prediction missing
                    return 0, 0, 1  # FN
            else:
                if pred_present:
                    # Prediction made but nothing in gold
                    return 0, 1, 0  # FP
                # Neither present: do nothing
                return 0, 0, 0
    
        # Update metrics for "Comment"
        tp, fp, fn = update_metrics(gold_comment, pred_comment)
        entity_stats[entity]["TP"] += tp
        entity_stats[entity]["FP"] += fp
        entity_stats[entity]["FN"] += fn
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
        # Update metrics for "Exact"
        tp, fp, fn = update_metrics(gold_exact, pred_exact)
        entity_stats[entity]["TP"] += tp
        entity_stats[entity]["FP"] += fp
        entity_stats[entity]["FN"] += fn
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    # Print per-entity metrics
    # print("Per-Entity Metrics:\n")
    # for entity in ENTITY_TYPES:
    #     stats = entity_stats[entity]
    #     precision, recall, f1 = compute_metrics_from_counts(stats["TP"], stats["FP"], stats["FN"])
    #     print(f"{entity:25s} P={precision:.3f}  R={recall:.3f}  F1={f1:.3f}  (TP={stats['TP']}, FP={stats['FP']}, FN={stats['FN']})")
    
# Compute overall micro-averaged metrics
overall_precision, overall_recall, overall_f1 = compute_metrics_from_counts(total_tp, total_fp, total_fn)
print("\nOverall (Micro-Averaged) Metrics:")
print(f"Precision = {overall_precision:.3f}")
print(f"Recall    = {overall_recall:.3f}")
print(f"F1        = {overall_f1:.3f}")
    
    


Error parsing answer YAML: mapping values are not allowed here
  in "<unicode string>", line 2, column 10:
      Comment: '[2024] EWCA Crim 474'
             ^
Error parsing gold YAML: while parsing a block mapping
  in "<unicode string>", line 2, column 3:
      Comment: "nan"
      ^
expected <block end>, but found '<scalar>'
  in "<unicode string>", line 3, column 19:
      Exact: "[2024]" EWCA Crim 474:judges
                      ^
Error parsing answer YAML: mapping values are not allowed here
  in "<unicode string>", line 2, column 10:
      Comment: '[2014] EWCA Crim 1594'
             ^
Error parsing gold YAML: while parsing a block mapping
  in "<unicode string>", line 2, column 3:
      Comment: "nan"
      ^
expected <block end>, but found '<scalar>'
  in "<unicode string>", line 3, column 19:
      Exact: "[2014]" EWCA Crim 1594:judges
                      ^
Error parsing answer YAML: mapping values are not allowed here
  in "<unicode string>", line 2, column 10:
      Com

In [11]:
import pandas as pd

# Prepare data structure for storing results per citation
results = {}

# Process each record in data
for record in data:
    answer_text = record.get("answer", "")
    gold_text = record.get("gold", "")

    # Parse YAML content from answer and gold fields
    try:
        answer_yaml = parse_yaml_content(answer_text) or {}
    except Exception as e:
        print(f"Error parsing answer YAML: {e}")
        answer_yaml = {}
    try:
        gold_yaml = parse_yaml_content(gold_text) or {}
    except Exception as e:
        print(f"Error parsing gold YAML: {e}")
        gold_yaml = {}

    # Extract citation for column grouping
    citation_number = gold_yaml.get("citation", {}).get("Exact", "Unknown Citation")

    if citation_number not in results:
        results[citation_number] = []

    for entity in ENTITY_TYPES:
        # Extract gold and predicted values for exact and comment
        gold_entity = gold_yaml.get(entity, {}) if isinstance(gold_yaml, dict) else {}
        pred_entity = answer_yaml.get(entity, {}) if isinstance(answer_yaml, dict) else {}

        gold_exact = gold_entity.get("Exact", "nan")
        gold_comment = gold_entity.get("Comment", "nan")
        pred_exact = pred_entity.get("Exact", "nan")
        pred_comment = pred_entity.get("Comment", "nan")

        # Combine exact and comment for display in a single cell
        human_combined = f"EXACT: {gold_exact}\nCOMMENT: {gold_comment}"
        ai_combined = f"EXACT: {pred_exact}\nCOMMENT: {pred_comment}"

        # Add results to the list for this citation
        results[citation_number].append({
            "Entity Type": entity,
            "Human": human_combined,
            "AI": ai_combined,
        })

# Create a DataFrame for structured evaluation output
rows = []
columns = []
for citation, entities in results.items():
    for entity_data in entities:
        entity_type = entity_data["Entity Type"]
        human_value = entity_data["Human"]
        ai_value = entity_data["AI"]

        # Initialize a row for the entity type if not already present
        if len(rows) < len(ENTITY_TYPES):
            rows.append({"Entity Type": entity_type})

        # Add human and AI data under the citation's subcolumns
        rows[ENTITY_TYPES.index(entity_type)].update({
            f"{citation} - Human": human_value,
            f"{citation} - AI": ai_value,
        })

# Convert to DataFrame and save as CSV
df_citation_eval = pd.DataFrame(rows)
output_csv_path = "data/evaluation_by_citation.csv"
df_citation_eval.to_csv(output_csv_path, index=False)

output_csv_path


Error parsing answer YAML: while parsing a block mapping
  in "<unicode string>", line 3, column 3:
      Comment: [2017] EWCA 849 (Crim)
      ^
expected <block end>, but found '<scalar>'
  in "<unicode string>", line 3, column 19:
      Comment: [2017] EWCA 849 (Crim)
                      ^
Error parsing answer YAML: while scanning a simple key
  in "<unicode string>", line 92, column 3:
      Exact:'seventy-five were of chil ... 
      ^
could not find expected ':'
  in "<unicode string>", line 94, column 1:
    VicSex:
    ^
Error parsing answer YAML: while parsing a block mapping
  in "<unicode string>", line 3, column 3:
      Comment: [2004] EWCA Crim 1528
      ^
expected <block end>, but found '<scalar>'
  in "<unicode string>", line 3, column 19:
      Comment: [2004] EWCA Crim 1528
                      ^
Error parsing answer YAML: while parsing a block mapping
  in "<unicode string>", line 3, column 3:
      Comment: [2007] EWCA Crim 2215
      ^
expected <block end>, but 

'data/evaluation_by_citation.csv'

In [12]:
df_citation_eval

,Entity Type,[2024] EWCA Crim 474 - Human,[2024] EWCA Crim 474 - AI,[2014] EWCA Crim 1594 - Human,[2014] EWCA Crim 1594 - AI,[2017] EWCA Crim 849 - Human,[2017] EWCA Crim 849 - AI,[2015] EWCA Crim 1791 - Human,[2015] EWCA Crim 1791 - AI,[2007] EWCA Crim 3312 - Human,...,[2016] EWCA Crim 1841 - Human,[2016] EWCA Crim 1841 - AI,[2019] EWCA Crim 1628 - Human,[2019] EWCA Crim 1628 - AI,[2007] EWCA Crim 2787 - Human,[2007] EWCA Crim 2787 - AI,[2004] EWCA Crim 2901 - Human,[2004] EWCA Crim 2901 - AI,[2018] EWCA Crim 1374 - Human,[2018] EWCA Crim 1374 - AI
0,citation,EXACT: [2024] EWCA Crim 474\nCOMMENT: nan,"EXACT: On 24 April 2024, Before: LORD JUSTICE ...",EXACT: [2014] EWCA Crim 1594\nCOMMENT: nan,"EXACT: On 12th June 2014, this court, differen...",EXACT: [2017] EWCA Crim 849\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: [2015] EWCA Crim 1791\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: [2007] EWCA Crim 3312\nCOMMENT: nan,...,EXACT: [2016] EWCA Crim 1841\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: [2019] EWCA Crim 1628\nCOMMENT: nan,EXACT: NCN: [2019] EWCA (Crim) 1628\nCOMMENT: ...,EXACT: [2007] EWCA Crim 2787\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: [2004] EWCA Crim 2901\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: [2018] EWCA Crim 1374\nCOMMENT: nan,"EXACT: On 22 January 2018, the appellant, Chri..."
1,judges,"EXACT: [''LORD JUSTICE MALES'', ''MR JUSTICE H...",EXACT: Before: LORD JUSTICE MALES MR JUSTICE H...,"EXACT: [''MR JUSTICE GLOBE'', ''SIR RODERICK E...",EXACT: PRESIDENT OF THE QUEEN\'S BENCH DIVISIO...,"EXACT: [''LORD JUSTICE HICKINBOTTOM'', ''MR JU...",EXACT: nan\nCOMMENT: nan,"EXACT: [''LADY JUSTICE RAFFERTY'', ''MR JUSTIC...",EXACT: nan\nCOMMENT: nan,"EXACT: [''LORD JUSTICE LAWS'', ''MR JUSTICE MA...",...,EXACT: [''MR JUSTICE MALES'']\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,"EXACT: [''LORD JUSTICE SIMON'', ''MRS JUSTICE ...",EXACT: B e f o r e : LORD JUSTICE SIMON MRS JU...,"EXACT: [''LORD JUSTICE LATHAM'', ''MR JUSTICE ...",EXACT: nan\nCOMMENT: nan,"EXACT: [''LORD JUSTICE CLARKE'', ''MR JUSTICE ...",EXACT: nan\nCOMMENT: nan,"EXACT: [''LORD JUSTICE SIMON'', ''MR JUSTICE G...",EXACT: B e f o r e : LORD JUSTICE SIMON MR JUS...
2,ConvCourtName,EXACT: nan\nCOMMENT: nan,"EXACT: On 18 October 2022, in the Crown Court ...",EXACT: nan\nCOMMENT: nan,EXACT: On 14th June 2011 in the Crown Court at...,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,...,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: On 8 March 2019 the appellant pleaded g...,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: He was sentenced in the Crown Court at ...
3,ConvictPleaDate,EXACT: nan\nCOMMENT: nan,"EXACT: On 18 October 2022, in the Crown Court ...",EXACT: nan\nCOMMENT: nan,EXACT: On 14th June 2011 in the Crown Court at...,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,...,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: On 8 March 2019 the appellant pleaded g...,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: On 22 January 2018\nCOMMENT: 2018-01-22
4,ConvictOffence,EXACT: nan\nCOMMENT: nan,"EXACT: possessing MDMA (count 2), possessing c...",EXACT: nan\nCOMMENT: nan,"EXACT: the applicant, Vincent Graham, pleaded ...",EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,...,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: pleaded guilty to two sexual offences i...,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: nan\nCOMMENT: nan,EXACT: He had pleaded guilty to drugs offences...
5,AcquitOffence,EXACT: nan\